Reading CSV File

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import torch
df = pd.read_csv("./Final_IC.csv")
df.head(5)

,article_id,article_desc
0,Article 1 of Indian Constitution,"Name and territory of the Union India, that is..."
1,Article 2 of Indian Constitution,Admission or establishment of new States: Parl...
2,Article 2A of Indian Constitution,Sikkim to be associated with the Union Rep by ...
3,Article 3 of Indian Constitution,Formation of new States and alteration of area...
4,Article 4 of Indian Constitution,Laws made under Articles 2 and 3 to provide fo...


In [3]:
df.head(5)

,article_id,article_desc
0,Article 1 of Indian Constitution,"Name and territory of the Union India, that is..."
1,Article 2 of Indian Constitution,Admission or establishment of new States: Parl...
2,Article 2A of Indian Constitution,Sikkim to be associated with the Union Rep by ...
3,Article 3 of Indian Constitution,Formation of new States and alteration of area...
4,Article 4 of Indian Constitution,Laws made under Articles 2 and 3 to provide fo...


In [4]:
df['ques'] = 'What is ' + df['article_id'].astype(str) + '?'
df['ans'] = df['article_id'].astype(str) + ' states that ' + df['article_desc']
df.drop('article_id',inplace=True,axis=1)
df.drop('article_desc',inplace=True,axis = 1)

In [5]:
import tensorflow as tf
Tokenizer = tf.keras.preprocessing.text.Tokenizer
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences
Sequential = tf.keras.models.Sequential
Embedding = tf.keras.layers.Embedding
SimpleRNN = tf.keras.layers.SimpleRNN
Dense = tf.keras.layers.Dense
LSTM = tf.keras.layers.LSTM
Dropout = tf.keras.layers.Dropout

In [9]:
questions = df['ques'].tolist()
answers = df['ans'].tolist()

In [10]:
tokenizer = Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts(answers)

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(answers)[0]

# Prepare input and target sequences
input_sequences = []
output_sequences = []

sequence_length = 100
for i in range(len(sequences) - sequence_length):
    input_sequences.append(sequences[i:i + sequence_length])
    output_sequences.append(sequences[i + sequence_length])

input_sequences = np.array(input_sequences)
output_sequences = np.array(output_sequences)

vocab_size = len(tokenizer.word_index) + 1

In [9]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=sequence_length),
    LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(vocab_size, activation="softmax"),
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 32)           1408      
                                                                 
 lstm (LSTM)                 (None, 100, 128)          82432     
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 44)                5676      
                                                                 
Total params: 221100 (863.67 KB)
Trainable params: 221100 (863.67 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
epochs = 200
batch_size = 32
model.fit(input_sequences, output_sequences, epochs=epochs, batch_size=batch_size)

Epoch 1/200
9/9 [==============================] - 10s 965ms/step - loss: 0.0970 - accuracy: 0.9856
Epoch 2/200
9/9 [==============================] - 10s 1s/step - loss: 0.0878 - accuracy: 0.9928
Epoch 3/200
9/9 [==============================] - 9s 863ms/step - loss: 0.0870 - accuracy: 0.9928
Epoch 4/200
9/9 [==============================] - 10s 1s/step - loss: 0.0865 - accuracy: 0.9892
Epoch 5/200
9/9 [==============================] - 8s 820ms/step - loss: 0.0820 - accuracy: 0.9892
Epoch 6/200
9/9 [==============================] - 11s 1s/step - loss: 0.0939 - accuracy: 0.9892
Epoch 7/200
9/9 [==============================] - 8s 804ms/step - loss: 0.0856 - accuracy: 0.9928
Epoch 8/200
9/9 [==============================] - 11s 1s/step - loss: 0.0855 - accuracy: 0.9928
Epoch 9/200
9/9 [==============================] - 7s 791ms/step - loss: 0.0823 - accuracy: 0.9820
Epoch 10/200
9/9 [==============================] - 12s 1s/step - loss: 0.0670 - accuracy: 0.9964
Epoch 11/200
9/9 [

In [7]:
def generate_text(seed_text, model, tokenizer, sequence_length, num_chars_to_generate):
    generated_text = seed_text

    for _ in range(num_chars_to_generate):
        token_list = tokenizer.texts_to_sequences([generated_text])
        token_list = pad_sequences(token_list, maxlen=sequence_length, padding="pre")
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_token = np.argmax(predicted_probs, axis=-1)[0]  # Get the index of the predicted token

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_token:
                output_word = word
                break

        generated_text += output_word

    return generated_text


In [19]:
model.save("partially_trained.keras")

In [17]:
seed_text = "Article 3 of Indian Constitution"

generated_text = generate_text(seed_text, model, tokenizer, sequence_length, num_chars_to_generate=250)
print(generated_text)

Article 3 of Indian Constitution territories thereof shall be as specified in the first schedule the territories specified in the first schedule; and such other territories as may be acquired in the first schedule the territories specified in the first schedule; and such other terr


Use trained Model

In [14]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# Load the pretrained model
loaded_model = load_model("./Pre-trained-Keras/partially_trained.h5")

# Use the generate_text function for prediction
seed_text = "Article 6 of Indian Constitution"
num_chars_to_generate = 250
generated_text = generate_text(seed_text, loaded_model, tokenizer, sequence_length, num_chars_to_generate)
print(generated_text)

Article 6 of Indian Constitution territories thereof shall be as specified in the first schedule the territories specified in the first schedule; and such other territories as may be acquired in the first schedule the territories specified in the first schedule; and such other terr
